# Delta Lake 0.7+ feature workarounds
Azure Synapse Analytics currently runs a fork of Delta Lake 0.6.x, which does not support all SQL commands and features available in Delta Lake 0.7+. This notebook contains Scala workarounds for these commands and features.


In [ ]:
// Generate some test data.
val df = spark.sql("SELECT 'foo' as Col1, 'bar' as Col2")

## Creating managed tables (with or without partitions)


In [ ]:
spark.sql("DROP TABLE IF EXISTS ManagedDeltaTable")
spark.sql("DROP TABLE IF EXISTS ExternalDeltaTable")

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// CREATE TABLE tableName USING DELTA

df.write.
    format("delta").
    saveAsTable("ManagedDeltaTable")

In [ ]:
val externalTablePath = "/tutorial/delta/externaltable"
df.write.
    format("delta").
    mode("overwrite").
    save(externalTablePath)
spark.sql(s"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '$externalTablePath'")

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// CREATE TABLE tableName USING DELTA PARTITIONED BY (...)

df.write.
  format("delta").
  mode("append").
  partitionBy("Col1").
  option("__partition_columns", """["Col1"]""").
  saveAsTable("PartitionedManagedDeltaTable")

## Reading from a storage path


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// SELECT * FROM delta.`/path/`

spark.read.
    format("delta").
    load(externalTablePath).
    show

## Inserting from one table into another


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// INSERT INTO table1 SELECT * FROM table2

spark.sql("SELECT * FROM ManagedDeltaTable").
    write.
    format("delta").
    mode("append").
    save(externalTablePath)

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// INSERT OVERWRITE table1 SELECT * FROM table2

spark.sql("SELECT * FROM ManagedDeltaTable").
    write.
    format("delta").
    mode("overwrite").
    save(externalTablePath)

## Updating or deleting rows from a table


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// DELETE FROM tableName WHERE (...)

import io.delta.tables._

val dt = DeltaTable.forPath(externalTablePath)

dt.delete("Col1 == 'foo'")

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// UPDATE tableName SET (...)

val describeExtended = spark.sql("DESCRIBE EXTENDED ManagedDeltaTable")
display(describeExtended)

In [ ]:
// Get the path to the table in storage.
val managedTablePath = describeExtended.
    where("col_name == 'Location'").
    select("data_type").
    collectAsList().
    get(0).
    getString(0)

// Construct the DeltaTable object from the path.
val managedTable = DeltaTable.forPath(managedTablePath)

// Run the update command.
managedTable.update(
        condition = expr("Col1 == 'foo'"),
        set = Map("Col2" -> lit("foobar")))

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// UPDATE delta.`/path/` WHERE (...)
DeltaTable.forPath(externalTablePath).
    update(
        condition = expr("Col1 == 'foo'"),
        set = Map("Col2" -> lit("foobar")))

## Merging two tables


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// MERGE INTO table1
// USING table2
// ON (...)
// WHEN MATCHED THEN (...)
// WHEN NOT MATCHED THEN (...)

DeltaTable.forPath(externalTablePath).as("ExternalTable").
  merge(managedTable.as("ManagedTable").toDF, "ExternalTable.Col1 == ManagedTable.Col1").
  whenMatched.
    update(Map("Col2" -> lit("This row matched."))).
  whenNotMatched.
    insert(Map("Col2" -> lit("This row did not match."))).
  execute()



## Changing the schema of a managed table.


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// ALTER TABLE tableName ADD COLUMNS (...)
// ALTER TABLE tableName CHANGE COLUMN (...)
// ALTER TABLE tableName REPLACE COLUMNS (...)

// Drop external table.
spark.sql("DROP TABLE ExternalDeltaTable")

// Reconfigure the table using DataFrame APIs...

// Recreate the table.
df.write.
    format("delta").
    mode("overwrite").
    save(externalTablePath)
spark.sql(s"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '$externalTablePath'")

## Configuring table properties


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// ALTER TABLE delta.`/path`
// SET TBLPROPERTIES(...)
// TBLPROPERTIES(
// delta.compatibility.symlinkFormatManifest.enabled=true)

// No workaround available.


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// TBLPROPERTIES(delta.logRetentionDuration = "interval <interval>")
// TBLPROPERTIES(delta.deletedFileRetentionDuration = "interval <interval>")

// Can only set these globally.
spark.conf.set("spark.databricks.delta.properties.defaults.delta.logRetentionDuration", "interval 2 days")
spark.conf.set("spark.databricks.delta.properties.defaults.delta.deletedFileRetentionDuration", "interval 1 days")

In [ ]:
// SET spark.databricks.delta.commitInfo.userMetadata=”{custom metadata}” INSERT …

// df.write.format("delta")
//   .mode(...)
//   .option("userMetadata", "{custom metadata}")
//   .save(...)

// No workaround available for these.


## DeltaTable.forName()


In [ ]:
// Delta Lake 0.7+ syntax: 
// 
// DeltaTable.forName(tableName)

val managedTablePath = spark.sql("DESCRIBE EXTENDED ManagedDeltaTable").
    where("col_name == 'Location'").
    select("data_type").
    collectAsList().
    get(0).
    getString(0)

DeltaTable.forPath(managedTablePath)